### __Estudo: ingestão de dados em geojson__

O presente notebook parte da necessidade de se inserir dados de análises autorais em arquivos `.geojson` ou `.json` para criação de mapas interativos. O objetivo é extrair de base geográfica já existente os dados de geolocalização e somá-los a resultados de análises para, ao final, converte a nova base em `.geojson`.

Trabalhamos aqui com dois arquivos:

- `dados.csv`: análise feita por Rodolfo Viana sobre índice de mortalidade no trânsito, por cidade da região metropolitana;
- `rmsp.geojson`: base para criação de polígonos criada por Rodolfo Viana a partir dos dados do IBGE.

Cada etapa do processo será comentada.

In [1]:
# Importação das bibliotecas
import json
import pandas as pd

# Conversão das bases em dataframe
data = pd.read_csv('dados.csv')
geodata = pd.read_json('rmsp.geojson')

In [2]:
# Checagem da base de dados
'''
Cabe destacar que a análise não consta neste estudo, tendo sido realizada anteriormente.
Na análise, os decimais foram configurados para três casas, e a delimitador foi a vírgula.
'''
data.head()

,Município,Mortos em 2019,Estimativa de habitantes,Mortos a cada 10 mil habitantes
0,ARUJÁ,8,88455,0.904
1,BARUERI,15,271306,0.553
2,BIRITIBA-MIRIM,3,32251,0.930
3,CAIEIRAS,7,100129,0.699
4,CAJAMAR,3,75638,0.397


In [3]:
# Checagem da base de geo
'''
Durante a criação do `.geojson`, quando se constatou que os dados formavam polígonos, manteve-se o padrão a seguir:

{
  "type": "FeatureCollection",
  "features": [
    "type": "Feature",
    "geometry": {
      "type": "Polygon",
      "coordinates": [[[...], [...], [...]]]
    }, 
    "properties": {"key": value, "key_2": value_2}
    }
  ]
}

Esse padrão, semelhante a um `dict` aninhado, tem dois nodes principais: 

- "type", cujos valores-padrão em `.geojson` são sempre "FeatureCollection";
- "features", onde devem entrar três keys obrigatórias: 
    - "type" (cujo value sempre será "Feature"), 
    - "geometry" (com o tipo de marcação geográfica (ponto, linha, polígono e afins), além das coordenadas),
    - "properties" (com quaisquer dados que queiramos apresentar no mapa)

Assim, temos de manipular "properties" sem alterar "geometry".
'''
geodata.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."


In [4]:
# Criação de listas para acomodar "features" e "coordinates"
feat = list()
coord = list()

for x in geodata['features']:
    for v in x['properties'].items():
        feat.append(v)

for i in geodata['features']:
    for v in i['geometry'].items():
        coord.append(v)

In [5]:
# Criação de dataframe vazio
file = pd.DataFrame()

'''
Como vamos usar os dados da análise agregados às coordenadas do `.geojson`, temos de ter um valor comum 
em ambas as bases que funcione como `primary key` na hora de fundir os dataframes.

Observamos que essa chave primária é `Município` na base de dados e `NM_MUNICIP` na base geolocalizada.

Assim, do `.geojson`, temos de extrair não apenas as coordenadas, mas os nomes dos municípios.
'''

# Criação de listas vazias
municipio = []
coordinates = []

# Iteração
for i in feat:
    if i[0] == 'NM_MUNICIP':
        municipio.append(i[1])
    else:
        pass

for i in coord:
    if i[0] == 'coordinates':
        coordinates.append(i[1][0])
    else:
        pass

# Transformação das listas em `series`
municipio = pd.Series(municipio)
coordinates = pd.Series(coordinates)

# Concatenação das séries em dataframe
geo = pd.concat([municipio, coordinates], axis=1).reset_index()

In [6]:
# Observação
geo.head()

,index,0,1
0,0,ARUJÁ,"[[-46.247678, -23.40562099968841], [-46.247911..."
1,1,BARUERI,"[[-46.799527999999995, -23.471382999689936], [..."
2,2,BIRITIBA-MIRIM,"[[-46.07703900000001, -23.514674999690882], [-..."
3,3,CAIEIRAS,"[[-46.670186999999935, -23.36309599968751], [-..."
4,4,CAJAMAR,"[[-46.83806400000001, -23.287843999685748], [-..."


In [7]:
# Renomeação e limpeza
geo.rename(columns={0:'municipio',1:'coordinates'}, inplace=True)
geo.drop(columns='index', inplace=True)

In [8]:
# Fusão dos dados da análise com os dados geolocalizados a partir de `Município` e `municipio`
df = pd.merge(data, geo, left_on='Município', right_on='municipio', how='inner')

In [9]:
# Observação
df.head()

,Município,Mortos em 2019,Estimativa de habitantes,Mortos a cada 10 mil habitantes,municipio,coordinates
0,ARUJÁ,8,88455,0.904,ARUJÁ,"[[-46.247678, -23.40562099968841], [-46.247911..."
1,BARUERI,15,271306,0.553,BARUERI,"[[-46.799527999999995, -23.471382999689936], [..."
2,BIRITIBA-MIRIM,3,32251,0.930,BIRITIBA-MIRIM,"[[-46.07703900000001, -23.514674999690882], [-..."
3,CAIEIRAS,7,100129,0.699,CAIEIRAS,"[[-46.670186999999935, -23.36309599968751], [-..."
4,CAJAMAR,3,75638,0.397,CAJAMAR,"[[-46.83806400000001, -23.287843999685748], [-..."


In [10]:
# Remoção da coluna `municipio`
df.drop(columns='municipio', inplace=True)

In [11]:
# Criação de função para formatar em `.geojson`
'''
Aqui recriamos o `.geojson` a partir do dataframe `df`: da primeira à penúltima coluna do dataframe temos "properties";
A última é "coordinates".

A estrutura é remontada seguindo o padrão de `.geojson`.
'''
def df2geojson(df, properties, coo='coordinates'):
    '''
    Args:
    df: dataframe
    properties: "properties" (neste caso, nome da cidade, quantidade de mortos, índice por 10 mil habitantes etc.)
    coo: coordenadas, latlong dos pontos do polígonos (default: 'coordinates')
    '''
    geojson = {"type":"FeatureCollection","features":[]}
    for _, row in df.iterrows():
        feature = {"type":"Feature",
                   "properties":{},
                   "geometry":{"type":"Polygon","coordinates":[]}}
        feature['geometry']['coordinates'] = [row[coo]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [12]:
# Seleção de colunas que devem entrar em "properties"
cols_prop = ['Município', 'Mortos em 2019', 'Estimativa de habitantes', 'Mortos a cada 10 mil habitantes']

In [13]:
# Chamada da função
geo_final = df2geojson(df, cols_prop)

In [14]:
# Observação
geo_final

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'Município': 'ARUJÁ',
    'Mortos em 2019': 8,
    'Estimativa de habitantes': 88455,
    'Mortos a cada 10 mil habitantes': 0.904},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-46.247678, -23.40562099968841],
      [-46.24791195299999, -23.405767410688448],
      [-46.248088999999986, -23.405879999688437],
      [-46.24841018600001, -23.406309110688504],
      [-46.248762825999876, -23.406592591688508],
      [-46.24892578700003, -23.4067234266885],
      [-46.249142232000004, -23.406902774688504],
      [-46.24945229499999, -23.407169464688458],
      [-46.249546947000006, -23.407366914688538],
      [-46.249565473000025, -23.407645723688518],
      [-46.24960429000004, -23.407826491688553],
      [-46.24976752899996, -23.407986687688485],
      [-46.249961686999946, -23.40805120268849],
      [-46.25027380600002, -23.40805283068854],
      [-46.250450404999974, -23.408142725688514],
      

In [15]:
# Dumping
geojson = json.dumps(geo_final, separators=(',', ':'))

In [16]:
# Salvamento em arquivo
output = 'mortos_transito.geojson'
with open(output, 'w') as file:
    file.write(geojson)

O resultado (mapa renderizado com os dados anexados) pode ser visto aqui: https://github.com/rodolfo-viana/dailylog/blob/master/estudos/self/mortos_transito.geojson